<a href="https://colab.research.google.com/github/WeronaF/analiza-tagow-SOW/blob/main/analiza_tagow_lubimyczytac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import regex as re
from collections import Counter
from itertools import chain
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [ ]:
data = "/content/sci-fi-data.csv" #plik z danymi głównymi
data_refined = "/content/sci-fi-refined.csv" #plik z danymi po edycji w OpenRefine
kategoryzacja = pd.read_csv("/content/sci-fi-kategoryzacja.csv")

df_main = pd.read_csv(data)
df_refined = pd.read_csv(data_refined)

In [ ]:
!mkdir wykresy
!mkdir data-csv

# Dane podstawowe

Liczba pustych rekordów

In [ ]:
nan_values = df_main["tagi"].isna().value_counts()
indexo = ('Książki z tagami', 'Książki bez tagów')
nan_values.index = indexo
nan_values

Wykres liczby książek z tagami i bez

In [ ]:
nan_values.plot.bar(color = 'black', figsize = (10,5))
plt.xticks(rotation = 0, fontsize = 20)
plt.yticks(fontsize = 20)

for index, value in enumerate(nan_values):
  if value !=10:
    plt.text(index - 0.07, value + 1, str(int(value/nan_values.sum()*100)) + '%', fontsize = 20)

plt.savefig("/content/wykresy/ilosc_ksiazek_z_tagami.png", transparent = True, bbox_inches = 'tight')

Funkcja do dodawania liczba nad słupkami w matplotlib.
* zbior - zmienna, z której tworzymy wykres
* wartosc1 - manipuluje ustawieniem liczb jednocyfrowych
* wartosc2 - manipuluje ustawieniem liczb dwucyfrowych
* wartosc3 - manipuluje ustawieniem liczb trzycyfrowych

In [ ]:
def liczby_nad_slupkami(zbior, wartosc1, wartosc2, wartosc3, font):
  for index, value in enumerate(zbior):
    if value < 10:
      index = index - wartosc1
    elif value > 99:
      index = index - wartosc3
    else:
      index = index - wartosc2
    plt.text(index, value + 0.2, str(value), fontsize = font)

Wykres top 5 autorów

In [ ]:
autorzy = df_main['autor'].value_counts().head(5)

autorzy.plot.bar(color = 'black', figsize = (10,5))
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)

liczby_nad_slupkami(autorzy, 0.1, 0.18, 0, 16)
plt.title("Top 5 autorów w zbiorze według liczby książek \n", fontsize = 16)

Rozkład ilościowy tagów po rozdzieleniu wartości przez *przecinek*/odpowiedni separator

In [ ]:
separator_main = ',\s+' # w razie potrzeby zmiana jedynie znaku separatora, "\s" zostawiamy
tagi_counts_list = df_main["tagi"].str.split(separator_main, expand=True).stack()

liczba_tagow = pd.DataFrame(tagi_counts_list.value_counts(), columns = ["liczba wystąpień"])
liczba_tagow

In [ ]:
# Jeśli chcemy zapisać sobie plik z ilością wystąpień danego tagu w biorze to uruchamiamy kod poniżej. Zapisanie w dokumencie csv.

liczba_tagow.to_csv('/content/data-csv/liczba_tagow.csv')

# Dane podstawowe- zbiór refined 

In [ ]:
kolumna_z_tagami_refined = "Index keywords" #nazwa kolumny z tagami w zbiorze refined
separator_refined = ";\s" # w razie potrzeby zmiana jedynie separatora, "\s" zostawiamy

df_refined[kolumna_z_tagami_refined].isna().value_counts()
tagi_counts_refined = df_refined[kolumna_z_tagami_refined].str.split(separator_refined, expand=True).stack().value_counts()

liczba_tagow_refined = pd.DataFrame(tagi_counts_refined, columns = ["liczba wystąpień"])
liczba_tagow_refined

# Jaki jest rozkład ilościowy tagów w danym zbiorze?

Wykres dla tagów df_main

In [ ]:
# creating a mask dla tagów, które wystąpiły więcej niż 1 raz

wystapienia = liczba_tagow["liczba wystąpień"] > 1
wystapienia_plot = liczba_tagow[wystapienia]

wystapienia_plot.plot.bar(width =0.6,
                           figsize = (16,6),
                           color = 'black',
                           alpha = 0.7,
                           ylabel = "liczba wystapien",
                           fontsize = 16)
plt.title(label = "Liczba wystąpień tagów wśród książek z kategorii science fiction \n", fontsize= 20)

plt.savefig('/content/wykresy/liczba-wystapien-tagow.png', transparent = True, bbox_inches = "tight")

Top 5 tagów

In [ ]:
wystapienia = liczba_tagow["liczba wystąpień"] > 1
wystapienia_plot = liczba_tagow[wystapienia].head(5)

wykres_wystapien = wystapienia_plot.plot.bar(width =0.6,
                           figsize = (16,6),
                           color = 'black',
                           alpha = 0.7,
                           ylabel = "liczba wystapien",
                           fontsize = 16)
plt.title(label = "/content/wykresy/Liczba wystąpień tagów wśród książek z kategorii science fiction \n", fontsize= 20)

plt.savefig('/content/wykresy/liczba-wystapien-tagow-top5.png', transparent = True, bbox_inches = "tight"  )

Wykres dla tagów df_refined

In [ ]:
wystapienia_refined = liczba_tagow_refined["liczba wystąpień"] > 1
wystapienia_plot_refined = liczba_tagow_refined[wystapienia_refined]

wykres_wystapien_refined = wystapienia_plot_refined.plot.bar(width =0.6,
                           figsize = (16,6),
                           color = 'black',
                           alpha = 0.7,
                           ylabel = "liczba wystapien",
                           fontsize = 16)
plt.title(label = "Liczba wystąpień tagów wśród książek z kategorii science fiction - po ujednoliceniu \n", fontsize= 20)

plt.savefig('/content/wykresy/liczba-wystapien-tagow-refined.png', transparent = True, bbox_inches = "tight"  )

Top 5 unikatowych tagów

In [ ]:
wystapienia_refined = liczba_tagow_refined["liczba wystąpień"] > 1
wystapienia_plot_refined = liczba_tagow_refined[wystapienia_refined].head(5)

wystapienia_plot_refined.plot.bar(width =0.6,
                           figsize = (16,6),
                           color = 'black',
                           alpha = 0.7,
                           ylabel = "liczba wystapien",
                           fontsize = 16)
plt.title(label = "Liczba wystąpień tagów wśród książek z kategorii science fiction - po ujednoliceniu \n", fontsize= 20)

plt.savefig('/content/wykresy/liczba-wystapien-tagow-refined-top5.png', transparent = True, bbox_inches = "tight")


# Kategoryzacja

In [ ]:
kategoryzacja

Tworzenie kolumny *kategorie tagów* w df_main

In [ ]:
frazy = kategoryzacja['tag']
kategorie = kategoryzacja['kategoria']
dict_cat = dict(zip(frazy, kategorie))

categories = []

for tag in df_main['tagi']:
  tag = str(tag).split(', ')
  category = []
  for fraza, rodzaj in dict_cat.items():
    for tago in tag:
      if tago == fraza:
        category.append(rodzaj)
      elif tago == 'nan':
        category.clear()
  categories.append(category)


categories = [str(s).replace('[', '') for s in categories]
categories = [str(s).replace(']', '') for s in categories]
categories = [str(s).replace("'", '') for s in categories]

df_main['kategorie tagów'] = categories
df_main

Ile tagów zawiera każada z występujących kategorii? - **tagi ogólnie**

In [ ]:
tagi_kategorie = df_main['kategorie tagów'].str.split(', ', expand=True).stack().value_counts()

tagi_kategorie.plot.bar(figsize = (15, 10),
                    width = 0.8,
                    color = 'black')
plt.xticks(rotation = 90, fontsize = 20)
plt.yticks(fontsize = 20)
liczby_nad_slupkami(tagi_kategorie, 0.1, 0.18, 0.18, 21)
plt.title(label = "Liczba tagów przypisana do kategorii \n", fontsize= 30)

plt.savefig('liczba-tagow-na-kategorie.png', transparent = True, bbox_inches = "tight")

Ile tagów zawiera każada z występujących kategorii? - **tagi unikatowe**

In [ ]:
wyniki_kat = kategorie.value_counts()

nazwy_kategorii = [values for values,key in wyniki_kat.items()]

wyniki_kat.plot.bar(figsize = (15, 10),
                    width = 0.8,
                    color = 'black')
plt.xticks(rotation = 90, fontsize = 20)
plt.yticks(fontsize = 20)
liczby_nad_slupkami(wyniki_kat, 0.1, 0.18, 0.18, 21) #dodawanie liczb nad słupkami z funkcji
plt.title(label = "Liczba unikatowych tagów przypisana do kategorii \n", fontsize= 30)

plt.savefig('liczba-tagow-unikatowych-na-kategorie.png', transparent = True, bbox_inches = "tight"  )

Jakie tagi należą do danej kategorii?

In [ ]:
for nazwa in nazwy_kategorii:
  warunek = kategoryzacja["kategoria"] == nazwa
  lista = [osoba for osoba in kategoryzacja[warunek]["tag"]]
  print(nazwa, "\n", lista)

# Jaka jest średnia liczba tagów na książkę?

In [ ]:
df_main['liczba tagów'] = [len(teksto.split(', ')) if len(teksto) != 0 else 0 for teksto in df_main['kategorie tagów']]

df_main.head(10)
df_main.to_csv('/content/data-csv/maindata.csv')

Wykres - wszystkie książki ze zbioru

In [ ]:
wykres_liczba_tagow  = pd.DataFrame(df_main['liczba tagów']).set_index(df_main["tytuł"])

wykres_liczba_tagow.plot.bar(figsize = (40, 20),
                             width = 0.8,
                             color = 'black',
                             alpha = 0.7)
plt.xticks(rotation = 90, fontsize = 20)
plt.yticks(fontsize = 25)
plt.title(label = "Liczba tagów przypisana do książki \n", fontsize= 40)
liczby_nad_slupkami(wykres_liczba_tagow['liczba tagów'], 0.1, 0.2, 0, 25)

#średnia i linia średniej na wykresie
srednia_tagow_sci_fi = df_main['liczba tagów'].mean()
plt.axhline(y = srednia_tagow_sci_fi, color='r', linestyle= '--', lw = 5)


plt.savefig('/content/wykresy/liczba-tagow-na-ksiazke.png', transparent = True, bbox_inches = "tight")


Wykres - tylko książki z tagami

In [ ]:
warunek_tagi = wykres_liczba_tagow['liczba tagów'] > 0
z_tagami = pd.DataFrame(wykres_liczba_tagow[warunek_tagi])

z_tagami.plot.bar(figsize = (35, 10),
                  width = 0.8,
                  color = 'black',
                  alpha = 0.7)
plt.xticks(rotation = 90, fontsize = 20)
plt.yticks(fontsize = 25)
plt.title(label = "Liczba tagów wśród książek z tagami \n", fontsize= 40)
liczby_nad_slupkami(z_tagami['liczba tagów'], 0.1, 0.2, 0, 25)

#średnia i linia średniej na wykresie
srednia_tagow_sci_fi_bez_zerowych = z_tagami['liczba tagów'].mean()
plt.axhline(y = srednia_tagow_sci_fi_bez_zerowych, color='r', linestyle='--', lw = 3)

plt.savefig('/content/wykresy/liczba-tagow-na-ksiazke_bez_zerowych.png', transparent = True, bbox_inches = "tight"  )


# Jak wygląda budowa tagów z punktu występowania wyrażeń wielowyrazowych?

Funkcja do obliczania liczby wyrazow w tagu z założeniem, że maksymalna liczba słów to 4.

In [ ]:
#kolumna_tagi jest listą lub pd.Series
def wielowyrazowosc(kolumna_tagi):
  jednowyrazowe = []
  dwuwyrazowe = []
  trzywyrazowe = []
  czterowyrazowe = []
  pieciowyrazowe = []

  for tag in kolumna_tagi:
    tag = tag.replace('- ','-') #łączenie myślnika z poprzednim słowem, aby myślnik nie był liczony jako słowo
    ile = tag.count(' ')
    if ile == 0:
      jednowyrazowe.append(tag)
    elif ile == 1:
      dwuwyrazowe.append(tag)
    elif ile == 2:
      trzywyrazowe.append(tag)
    elif ile == 3:
      czterowyrazowe.append(tag)
    elif ile == 4:
      pieciowyrazowe.append(tag)

  rodzaj_tagu = [jednowyrazowe, dwuwyrazowe, trzywyrazowe, czterowyrazowe, pieciowyrazowe]
  tagi_liczba_wyrazow = ["jednowyrazowe", "dwuwyrazowe", "trzywyrazowe", "czterowyrazowe", "pięciowyrazowe"]
  liczba_tagow_wielowyrazowosc = [len(jednowyrazowe), len(dwuwyrazowe), len(trzywyrazowe), len(czterowyrazowe), len(pieciowyrazowe)]

  wyrazenia_wielowyrazowe = pd.DataFrame(liczba_tagow_wielowyrazowosc, columns = ['liczba tagów'])
  wyrazenia_wielowyrazowe.insert(0,'liczba wyrazow w tagu', range(1,6))

  return pd.DataFrame(wyrazenia_wielowyrazowe['liczba tagów']).set_index(pd.Series(tagi_liczba_wyrazow))


In [ ]:
wszystkie_tagi = tagi_counts_list
tagi_unikatowe = liczba_tagow.index

wielowyrazowosc_wszystkie_tagi = wielowyrazowosc(wszystkie_tagi)
wielowyrazowosc_tagi_unikatowe = wielowyrazowosc(tagi_unikatowe)

print(f'''
Rozkład dla wszystkich tagów w zbiorze:

{wielowyrazowosc_wszystkie_tagi}

Rozkład dla tagów unikatowych w zbiorze:

{wielowyrazowosc_tagi_unikatowe}''')

In [ ]:
#kolumna_tagi jest listą lub pd.Series
type(liczba_tagow.index)
type(tagi_counts_list)

Wykres - wszystkie tagi

In [ ]:
wielowyrazowosc_wszystkie_tagi.plot.bar(figsize = (12,8),
                                         color = 'black')
plt.xticks(rotation = 10, fontsize = 15)
plt.yticks(fontsize = 15)
liczby_nad_slupkami(wielowyrazowosc_wszystkie_tagi['liczba tagów'], 0.05, 0.1, 0.15, 20)

plt.title(label = "Liczba tagów w zależności od liczby wyrazów \n", fontsize= 15)
plt.savefig('/content/wykresy/tagi-wyrazenia-wielowyrazowe.png', transparent = True, bbox_inches = "tight") #png zapisuje się automatycznie w plikach 

Wykres - tagi unikatowe

In [ ]:
wielowyrazowosc_tagi_unikatowe.plot.bar(figsize = (12,8),
                                        color = 'black')
plt.xticks(rotation = 10, fontsize = 15)
plt.yticks(fontsize = 15)
liczby_nad_slupkami(wielowyrazowosc_tagi_unikatowe['liczba tagów'], 0.05, 0.1, 0.15, 20)

plt.title(label = "Liczba tagów w zależności od liczby wyrazów dla unikatowych tagów \n", fontsize= 15)
plt.savefig('/content/wykresy/tagi-wyrazenia-wielowyrazowe-unikaty.png', transparent = True, bbox_inches = "tight")

# Jaki charakter mają tagi (nazwy własne, nazwy pospolite, skróty)?

In [ ]:
pattern_wlasne = "[[:upper:]][a-z]+"
pattern_skroty = "[[:upper:]]{2,}"

nazwy_wlasne = []
nazwy_pospolite = []
skroty = []

def charakter(dane):
  for tag in dane:
    if re.match(pattern_wlasne, str(tag)):
      nazwy_wlasne.append(tag)
    elif re.match(pattern_skroty, str(tag)):
      skroty.append(tag)
    else:
      nazwy_pospolite.append(tag)

  return print(f'''
  Nazwy własne: {len(nazwy_wlasne)}
  Nazwy pospolite: {len(nazwy_pospolite)}
  Skróty: {len(skroty)} {skroty}''')


In [ ]:
charakter(tagi_unikatowe)

In [ ]:
charakter(wszystkie_tagi)